In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import plotly.express as px
import pandas as pd
dataframe_nuevo = pd.read_csv("/content/drive/MyDrive/datos_limpios_pro.csv")

In [ ]:
dataframe_nuevo['fecha'] = pd.to_datetime(dataframe_nuevo['fecha'])

# Extraer el mes y el año de la columna 'fecha'
dataframe_nuevo['mes'] = dataframe_nuevo['fecha'].dt.to_period("M") #MES

# Agrupar por mes, descrip y otras columnas, y sumar las ventas
dataframe_nuevo = dataframe_nuevo.groupby(['mes', 'descrip', 'suc', 'nombre_color', 'design']).agg({'cantidad': 'sum'}).reset_index()

# Mostrar el nuevo DataFrame
dataframe_nuevo

,mes,descrip,suc,nombre_color,design,cantidad
0,2021-02,ACCESORIO CINTA TAPIZADO,2,VERDE,LISO,150.00
1,2021-02,ACCESORIO CINTA TAPIZADO,4,VERDE,LISO,100.00
2,2021-02,ACCESORIO CINTA TAPIZADO,5,AZUL,LISO,496.65
3,2021-02,ACCESORIO CINTA TAPIZADO,5,NEGRO METEORITO,LISO,300.00
4,2021-02,ACCESORIO CINTA TAPIZADO,5,VERDE,LISO,650.00
...,...,...,...,...,...,...
589114,2023-08,VIYELAS ESTAMPADA,6,SALMON CREMA,TARTAN,3.00
589115,2023-08,VIYELAS ESTAMPADA,10,AMARILLO MINERAL,FLORAL,1.00
589116,2023-08,VIYELAS ESTAMPADA,10,AZUL MEDIEVAL,TARTAN,1.50
589117,2023-08,VIYELAS ESTAMPADA,24,AMARILLO MINERAL,FLORAL,0.50


In [ ]:
# Renombrar la columna 'cantidad' a 'venta'
dataframe_nuevo = dataframe_nuevo.rename(columns={'cantidad': 'venta'})

In [ ]:
dataframe_nuevo['descrip'] = dataframe_nuevo['descrip'] + ' - ' + dataframe_nuevo['suc'].astype(str) + ' - ' + dataframe_nuevo['nombre_color']

# Luego, puedes eliminar las columnas originales si ya no las necesitas
dataframe_nuevo = dataframe_nuevo.drop(['suc', 'nombre_color'], axis=1)

In [ ]:
dataframe_nuevo = dataframe_nuevo.drop(['design'], axis=1)

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from tqdm import tqdm

def remove_non_stationary(df, significance_level=0.05, min_sample_size=10):
    df_stationary = pd.DataFrame(columns=['mes', 'descrip', 'venta'])
    removed_count = 0
    removal_criterion = []

    unique_products = df['descrip'].unique()

    for product in tqdm(unique_products, desc="Procesando productos"):
        df_product = df[df['descrip'] == product]

        # Verificar que el tamaño de la muestra sea suficientemente grande porque sino da error.
        if len(df_product) < min_sample_size:
            removed_count += len(df_product)
            removal_criterion.append(f"Producto '{product}': Tamaño de muestra insuficiente.")
            continue

        # Realizar la prueba de Dickey-Fuller aumentada
        result = adfuller(df_product['venta'])
        p_value = result[1]

        if p_value <= significance_level:
            df_stationary = pd.concat([df_stationary, df_product])
        else:
            removed_count += len(df_product)
            removal_criterion.append(f"Producto '{product}': p-value = {p_value} > {significance_level}")

    print(f"Se eliminaron {removed_count} filas no estacionarias.")
    print("Criterios de eliminación:")
    for criterion in removal_criterion:
        print(criterion)

    return df_stationary

df_stationary = remove_non_stationary(dataframe_nuevo)

# Ahora puedes usar df_stationary para el análisis de ARIMA


Procesando productos:  35%|███▍      | 24516/70986 [22:18<36:33, 21.19it/s]/usr/local/lib/python3.10/dist-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
Procesando productos: 100%|██████████| 70986/70986 [1:02:50<00:00, 18.83it/s]


Se han truncado las últimas 5000 líneas del flujo de salida.
Producto 'ENCAJE REBORDE - 5 - BEIGE NATURAL': Tamaño de muestra insuficiente.
Producto 'ENCAJE REBORDE - 5 - GRIS NEUTRO': Tamaño de muestra insuficiente.
Producto 'ENCAJE REBORDE - 6 - ROSA PARFAIT': p-value = 0.09453886877482481 > 0.05
Producto 'ENCAJE REBORDE - 10 - LILA EASTER': p-value = 0.9929657943478756 > 0.05
Producto 'ENCAJE REBORDE - 10 - LILA ZARZAMORA': Tamaño de muestra insuficiente.
Producto 'ENCAJE REBORDE - 10 - ROSA AMANECER ARENA': Tamaño de muestra insuficiente.
Producto 'ENCAJE REBORDE - 10 - ROSA PARFAIT': Tamaño de muestra insuficiente.
Producto 'ENCAJE REBORDE - 24 - BLANCO ARENA': Tamaño de muestra insuficiente.
Producto 'ENCAJE REBORDE - 24 - FUCSIA ORQUIDEA SALVAJE': p-value = 0.42279164068808517 > 0.05
Producto 'ENCAJE REBORDE - 25 - ROSA PARFAIT': Tamaño de muestra insuficiente.
Producto 'ENCAJE REBORDE - 30 - ROSA AMANECER ARENA': Tamaño de muestra insuficiente.
Producto 'ENCAJE REBORDE - 30 - R

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
df_stationary.rename(columns={'mes': 'fecha'}, inplace=True)

In [ ]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

# Establecer las fechas de inicio y fin con el formato de mes/año
fecha_inicio = pd.to_datetime('2023-09-01').to_period('M')
fecha_fin = pd.to_datetime('2024-09-01').to_period('M')

# Crear un rango de fechas mensuales
periodo_prediccion = pd.period_range(start=fecha_inicio, end=fecha_fin, freq='M')

# Crear un DataFrame para almacenar las predicciones
predicciones_df = pd.DataFrame(columns=['fecha', 'descrip', 'prediccion'])

# Iterar sobre cada artículo 'descrip'
for producto in df_stationary['descrip'].unique():
    # Filtrar el DataFrame por producto
    df_producto = df_stationary[df_stationary['descrip'] == producto]

    # Crear un modelo ARIMA para el producto actual
    modelo_arima = ARIMA(df_producto['venta'], order=(1, 0, 1))  # Puedes ajustar los parámetros del modelo según sea necesario

    # Ajustar el modelo
    modelo_entrenado = modelo_arima.fit()

    # Realizar predicciones para el periodo de septiembre 2023 a septiembre 2024
    predicciones = modelo_entrenado.get_forecast(steps=len(periodo_prediccion))
    predicciones_ci = predicciones.conf_int()

    # Almacenar las predicciones en el DataFrame de predicciones
    predicciones_df = pd.concat([predicciones_df, pd.DataFrame({
        'fecha': periodo_prediccion.astype(str),
        'descrip': producto,
        'prediccion': predicciones.predicted_mean.values
    })])

# Mostrar el DataFrame de predicciones


Se han truncado las últimas 5000 líneas del flujo de salida.
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No su

In [ ]:
predicciones_df

,fecha,descrip,prediccion
0,2023-09,ACCESORIO CINTA TAPIZADO - 2 - VERDE,426.799198
1,2023-10,ACCESORIO CINTA TAPIZADO - 2 - VERDE,461.074691
2,2023-11,ACCESORIO CINTA TAPIZADO - 2 - VERDE,472.615149
3,2023-12,ACCESORIO CINTA TAPIZADO - 2 - VERDE,476.500788
4,2024-01,ACCESORIO CINTA TAPIZADO - 2 - VERDE,477.809071
...,...,...,...
8,2024-05,POLAR INFANTIL - 4 - ROJO MARTE,7.217269
9,2024-06,POLAR INFANTIL - 4 - ROJO MARTE,7.215462
10,2024-07,POLAR INFANTIL - 4 - ROJO MARTE,7.216180
11,2024-08,POLAR INFANTIL - 4 - ROJO MARTE,7.215894
